In [1]:
# %matplotlib widget
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ['PYOPENGL_PLATFORM'] = 'egl'

from src.vector_object import *

from src.frame import Frame

from src.feature import SDF_Feature
from src.dataset import *
from src.utils import *
from src.data_gen_utils import *

from os import path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import trimesh

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
exp_name = 'PIFO_best'
state = torch.load('network/'+exp_name+'.pth.tar')
C = state['config']

trainset = PIFODataset(C['DATA_FILENAME'],
                       num_views=C['NUM_VIEWS'],
                       num_points=C['NUM_POINTS'],
                       num_grasps=C['NUM_GRASPS'],
                       num_hangs=C['NUM_HANGS'],
                       grasp_draw_points=C['GRASP_DRAW_POINTS'],
                       hang_draw_points=C['HANG_DRAW_POINTS'],
                       random_erase=False,
                       on_gpu_memory=True)

testset = PIFODataset('data/test_batch.hdf5',
                      num_views=C['NUM_VIEWS'],
                      num_points=C['NUM_POINTS'],
                      num_grasps=C['NUM_GRASPS'],
                      num_hangs=C['NUM_HANGS'],
                      grasp_draw_points=C['GRASP_DRAW_POINTS'],
                      hang_draw_points=C['HANG_DRAW_POINTS'],
                      random_erase=False,
                      on_gpu_memory=True)


warper = RandomImageWarper(img_res=C['IMG_RES'], 
                           sig_center=0, 
                           return_cam_params=True)

# PIFO
obj1 = Frame()
obj1.build_backbone(pretrained=True, **C)
obj1.build_sdf_head(C['SDF_HEAD_HIDDEN'])
obj1.build_keypoint_head('grasp', C['GRASP_HEAD_HIDDEN'], C['GRIPPER_POINTS'])
obj1.build_keypoint_head('hang', C['HANG_HEAD_HIDDEN'], C['HOOK_POINTS'])
obj1.load_state_dict(state['network'])
obj1.to(device).eval()
F_grasp1 = KeyPoint_Feature(obj1, 'grasp')
F_hang1 = KeyPoint_Feature(obj1, 'hang')

# noPixel
exp_name = 'noPixelAligned_best'
state = torch.load('network/'+exp_name+'.pth.tar')
C = state['config']
obj2 = Frame()
obj2.build_backbone(pretrained=True, **C)
obj2.build_sdf_head(C['SDF_HEAD_HIDDEN'])
obj2.build_keypoint_head('grasp', C['GRASP_HEAD_HIDDEN'], C['GRIPPER_POINTS'])
obj2.build_keypoint_head('hang', C['HANG_HEAD_HIDDEN'], C['HOOK_POINTS'])
obj2.load_state_dict(state['network'])
obj2.to(device).eval()
F_grasp2 = KeyPoint_Feature(obj2, 'grasp')
F_hang2 = KeyPoint_Feature(obj2, 'hang')

# vecObj
exp_name = 'vectorObject_best'
state = torch.load('network/'+exp_name+'.pth.tar')
C = state['config']
obj3 = Frame_vec()
obj3.build_backbone(pretrained=True, **C)
obj3.build_sdf_head(C['SDF_HEAD_HIDDEN'])
obj3.build_pose_head('grasp', C['GRASP_HEAD_HIDDEN'])
obj3.build_pose_head('hang', C['HANG_HEAD_HIDDEN'])
obj3.load_state_dict(state['network'])
obj3.to(device).eval()

F_grasp3 = Pose_Feature_vec(obj3, 'grasp')
F_hang3 = Pose_Feature_vec(obj3, 'hang')

In [3]:
for num_views in [2, 8]:
    print('======================== '+str(num_views)+' ========================')
    for t, dataset in enumerate([trainset, testset]):
        dataset.num_views = num_views


        x_PIFO, x_noPixel, x_vecObj = [], [], []
        y_PIFO, y_noPixel, y_vecObj = [], [], []
        B, N, num_best = len(dataset), 10, 1

        for it in range(10):
            rgb_list, projections_list, cam_params_list, filename_list, mass_list, com_list = [], [], [], [], [], []
            for i in range(B):
                data = to_device(dataset[i], device)
                rgb, projections, cam_pos, new_origin, cam_roll = warper(data['rgb'].unsqueeze(0), 
                                          data['cam_extrinsic'].unsqueeze(0), 
                                          data['cam_intrinsic'].unsqueeze(0))
                rgb_list.append(rgb)
                projections_list.append(projections)
                cam_params_list.append(torch.cat([cam_pos, cam_roll], dim=2))
                filename_list.append(data['filenames'])
                mass_list.append(data['masses'])
                com_list.append(data['coms'])

            x_init = torch.cat([.2*torch.randn(B,N,3, device=device), 
                               random_quaternions(B*N, device=device).view(B,N,4)], dim=2)

            x, cost, coll = F_hang1.optimize(x_init.clone(),
                                              torch.cat(rgb_list), 
                                              torch.cat(projections_list))

            best_inds = torch.tensor(cost).argsort(dim=1)[:, :num_best].to(device).view(B,num_best,1).expand(-1,-1,7)
            best_poses = torch.gather(x, dim=1, index=best_inds)

            x_PIFO.append(best_poses)

            x, cost, coll = F_hang1.optimize(x,
                                             torch.cat(rgb_list), 
                                             torch.cat(projections_list),
                                             w_coll=1e3,
                                             coll_margin=0)

            best_inds = torch.tensor(np.square(cost)+np.square(coll*1e3)).argsort(dim=1)[:, :num_best].to(device).view(B,num_best,1).expand(-1,-1,7)
            best_poses = torch.gather(x, dim=1, index=best_inds)

            y_PIFO.append(best_poses)


            ### 2
            x, cost, coll = F_hang2.optimize(x_init.clone(),
                                              torch.cat(rgb_list), 
                                              torch.cat(projections_list))

            best_inds = torch.tensor(cost).argsort(dim=1)[:, :num_best].to(device).view(B,num_best,1).expand(-1,-1,7)
            best_poses = torch.gather(x, dim=1, index=best_inds)

            x_noPixel.append(best_poses)

            x, cost, coll = F_hang2.optimize(x,
                                             torch.cat(rgb_list), 
                                             torch.cat(projections_list),
                                             w_coll=1e3,
                                             coll_margin=0)

            best_inds = torch.tensor(np.square(cost)+np.square(coll*1e3)).argsort(dim=1)[:, :num_best].to(device).view(B,num_best,1).expand(-1,-1,7)
            best_poses = torch.gather(x, dim=1, index=best_inds)

            y_noPixel.append(best_poses)


            ### 3
            x, cost, coll = F_hang3.optimize(x_init.clone(),
                                              torch.cat(rgb_list), 
                                              torch.cat(cam_params_list))

            best_inds = torch.tensor(cost).argsort(dim=1)[:, :num_best].to(device).view(B,num_best,1).expand(-1,-1,7)
            best_poses = torch.gather(x, dim=1, index=best_inds)

            x_vecObj.append(best_poses)

            x, cost, coll = F_hang3.optimize(x,
                                             torch.cat(rgb_list), 
                                             torch.cat(cam_params_list),
                                             w_coll=1e3,
                                             coll_margin=0)

            best_inds = torch.tensor(np.square(cost)+np.square(coll*1e3)).argsort(dim=1)[:, :num_best].to(device).view(B,num_best,1).expand(-1,-1,7)
            best_poses = torch.gather(x, dim=1, index=best_inds)

            y_vecObj.append(best_poses)

        f1_PIFO = F_hang1.check_feasibility(torch.cat(x_PIFO, dim=1),
                                             filename_list, 
                                             mass_list,
                                             com_list)
        f2_PIFO = F_hang1.check_feasibility(torch.cat(y_PIFO, dim=1),
                                             filename_list, 
                                             mass_list,
                                             com_list)

        f1_noPixel = F_hang2.check_feasibility(torch.cat(x_noPixel, dim=1),
                                             filename_list, 
                                             mass_list,
                                             com_list)
        f2_noPixel = F_hang2.check_feasibility(torch.cat(y_noPixel, dim=1),
                                             filename_list, 
                                             mass_list,
                                             com_list)

        f1_vecObj = F_hang3.check_feasibility(torch.cat(x_vecObj, dim=1),
                                             filename_list, 
                                             mass_list,
                                             com_list)
        f2_vecObj = F_hang3.check_feasibility(torch.cat(y_vecObj, dim=1),
                                             filename_list, 
                                             mass_list,
                                             com_list)

        print(f1_PIFO.sum()/f1_PIFO.size, f2_PIFO.sum()/f2_PIFO.size)
        print(f1_noPixel.sum()/f1_noPixel.size, f2_noPixel.sum()/f2_noPixel.size)
        print(f1_vecObj.sum()/f1_vecObj.size, f2_vecObj.sum()/f2_vecObj.size)


        data_name = 'train' if t == 0 else 'test'
        with h5py.File('evals/hang/'+data_name+'_'+str(num_views)+'.hdf5', mode='w') as f:
            f.create_dataset("x_PIFO", data=torch.cat(x_PIFO, dim=1).cpu().numpy())
            f.create_dataset("y_PIFO", data=torch.cat(y_PIFO, dim=1).cpu().numpy())

            f.create_dataset("x_noPixel", data=torch.cat(x_noPixel, dim=1).cpu().numpy())
            f.create_dataset("y_noPixel", data=torch.cat(y_noPixel, dim=1).cpu().numpy())

            f.create_dataset("x_vecObj", data=torch.cat(x_vecObj, dim=1).cpu().numpy())
            f.create_dataset("y_vecObj", data=torch.cat(y_vecObj, dim=1).cpu().numpy())
            
            f.create_dataset("f1_PIFO", data=f1_PIFO)
            f.create_dataset("f2_PIFO", data=f2_PIFO)

            f.create_dataset("f1_noPixel", data=f1_noPixel)
            f.create_dataset("f2_noPixel", data=f2_noPixel)

            f.create_dataset("f1_vecObj", data=f1_vecObj)
            f.create_dataset("f2_vecObj", data=f2_vecObj)

======================== 2 ========================
iter: 0, cost: 6.226168155670166, coll: 0.0
iter: 0, cost: 2.7627921104431152, coll: 623.4053344726562
iter: 0, cost: 6.349523544311523, coll: 0.0
iter: 0, cost: 2.683420419692993, coll: 407.5447692871094
iter: 0, cost: 5.754220485687256, coll: 0.0
iter: 0, cost: 2.7320165634155273, coll: 235.4281768798828
iter: 0, cost: 5.949152946472168, coll: 0.0
iter: 0, cost: 3.5128183364868164, coll: 853.0565185546875
iter: 0, cost: 5.521780014038086, coll: 0.0
iter: 0, cost: 2.280139684677124, coll: 539.4727172851562
iter: 0, cost: 6.237496852874756, coll: 0.0
iter: 0, cost: 2.563833713531494, coll: 341.11669921875
iter: 0, cost: 7.348587989807129, coll: 0.0
iter: 0, cost: 4.258683204650879, coll: 633.6900634765625
iter: 0, cost: 5.501636028289795, coll: 0.0
iter: 0, cost: 2.2472763061523438, coll: 457.487548828125
iter: 0, cost: 5.497444152832031, coll: 0.0
iter: 0, cost: 3.0263781547546387, coll: 301.6823425292969
iter: 0, cost: 6.68783903121

In [16]:
for num_views in [2, 4, 8]:
    print('======================== '+str(num_views)+' ========================')
    for t, dataset in enumerate([trainset, testset]):

        data_name = 'train' if t == 0 else 'test'

        with h5py.File('evals/hang/'+data_name+'_'+str(num_views)+'.hdf5', mode='r+') as f:
            del f['f1_PIFO']
            del f['f2_PIFO']
            del f['f1_noPixel']
            del f['f2_noPixel']
            del f['f1_vecObj']
            del f['f2_vecObj']

======================== 2 ========================


KeyError: "Couldn't delete link (callback link pointer is NULL (specified link may be '.' or not exist))"

In [17]:
for num_views in [2, 4, 8]:
    print('======================== '+str(num_views)+' ========================')
    for t, dataset in enumerate([trainset, testset]):
        dataset.num_views = num_views
        B, N, num_best = len(dataset), 10, 1

        data_name = 'train' if t == 0 else 'test'
        filename_list, mass_list, com_list = [], [], []
        for i in range(B):
            data = to_device(dataset[i], device)
            filename_list.append(data['filenames'])
            mass_list.append(data['masses'])
            com_list.append(data['coms'])
        
        with h5py.File('evals/hang/'+data_name+'_'+str(num_views)+'.hdf5', mode='r') as f:
            x_PIFO, x_noPixel, x_vecObj = f['x_PIFO'][:], f['x_noPixel'][:], f['x_vecObj'][:] 
            y_PIFO, y_noPixel, y_vecObj = f['y_PIFO'][:], f['y_noPixel'][:], f['y_vecObj'][:] 


        f1_PIFO = F_hang1.check_feasibility(torch.Tensor(x_PIFO),
                                             filename_list, 
                                             mass_list,
                                             com_list)
        f2_PIFO = F_hang1.check_feasibility(torch.Tensor(y_PIFO),
                                             filename_list, 
                                             mass_list,
                                             com_list)

        f1_noPixel = F_hang2.check_feasibility(torch.Tensor(x_noPixel),
                                             filename_list, 
                                             mass_list,
                                             com_list)
        f2_noPixel = F_hang2.check_feasibility(torch.Tensor(y_noPixel),
                                             filename_list, 
                                             mass_list,
                                             com_list)

        f1_vecObj = F_hang3.check_feasibility(torch.Tensor(x_vecObj),
                                             filename_list, 
                                             mass_list,
                                             com_list)
        f2_vecObj = F_hang3.check_feasibility(torch.Tensor(y_vecObj),
                                             filename_list, 
                                             mass_list,
                                             com_list)

        print(f1_PIFO.sum()/f1_PIFO.size, f2_PIFO.sum()/f2_PIFO.size)
        print(f1_noPixel.sum()/f1_noPixel.size, f2_noPixel.sum()/f2_noPixel.size)
        print(f1_vecObj.sum()/f1_vecObj.size, f2_vecObj.sum()/f2_vecObj.size)


        with h5py.File('evals/hang/'+data_name+'_'+str(num_views)+'.hdf5', mode='r+') as f:
          
            
            f.create_dataset("f1_PIFO", data=f1_PIFO)
            f.create_dataset("f2_PIFO", data=f2_PIFO)

            f.create_dataset("f1_noPixel", data=f1_noPixel)
            f.create_dataset("f2_noPixel", data=f2_noPixel)

            f.create_dataset("f1_vecObj", data=f1_vecObj)
            f.create_dataset("f2_vecObj", data=f2_vecObj)

======================== 2 ========================
0.8717948717948718 0.882051282051282
0.8833333333333333 0.8628205128205129
0.2564102564102564 0.0
0.7142857142857143 0.7214285714285714
0.7035714285714286 0.7178571428571429
0.21785714285714286 0.0
======================== 4 ========================
0.8871794871794871 0.9397435897435897
0.9025641025641026 0.9115384615384615
0.5512820512820513 0.0
0.7535714285714286 0.7892857142857143
0.7571428571428571 0.7821428571428571
0.4642857142857143 0.0
======================== 8 ========================
0.9166666666666666 0.9653846153846154
0.9128205128205128 0.9294871794871795
0.658974358974359 0.0
0.8035714285714286 0.825
0.775 0.8035714285714286
0.49642857142857144 0.0


In [19]:
for num_views in [2, 4, 8]:
    for data_name in ['train', 'test']:
        with h5py.File('evals/hang/'+data_name+'_'+str(num_views)+'.hdf5', mode='r') as f:
            f1_PIFO, f1_noPixel, f1_vecObj = f['f1_PIFO'][:], f['f1_noPixel'][:], f['f1_vecObj'][:] 
            f2_PIFO, f2_noPixel, f2_vecObj = f['f2_PIFO'][:], f['f2_noPixel'][:], f['f2_vecObj'][:] 
        print('======================== '+data_name+'_'+str(num_views)+' ========================')
        print(f1_PIFO.sum()/f1_PIFO.size, f2_PIFO.sum()/f2_PIFO.size)
        print(f1_noPixel.sum()/f1_noPixel.size, f2_noPixel.sum()/f2_noPixel.size)
        print(f1_vecObj.sum()/f1_vecObj.size, f2_vecObj.sum()/f2_vecObj.size)

======================== train_2 ========================
0.8717948717948718 0.882051282051282
0.8833333333333333 0.8628205128205129
0.2564102564102564 0.0
======================== test_2 ========================
0.7142857142857143 0.7214285714285714
0.7035714285714286 0.7178571428571429
0.21785714285714286 0.0
======================== train_4 ========================
0.8871794871794871 0.9397435897435897
0.9025641025641026 0.9115384615384615
0.5512820512820513 0.0
======================== test_4 ========================
0.7535714285714286 0.7892857142857143
0.7571428571428571 0.7821428571428571
0.4642857142857143 0.0
======================== train_8 ========================
0.9166666666666666 0.9653846153846154
0.9128205128205128 0.9294871794871795
0.658974358974359 0.0
======================== test_8 ========================
0.8035714285714286 0.825
0.775 0.8035714285714286
0.49642857142857144 0.0
